<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Compress" data-toc-modified-id="Compress-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Compress</a></span></li></ul></div>

To extract other elements, change the FOLDER.

In [19]:
import pandas as pd
import numpy as np
import pdb
import requests
import time
import random
import os
import glob
import sys
import shutil

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options

In [20]:
def get_all_files_in_a_folder(folder, full=False):
    path = "{}{}".format(SAVING_PATH, folder)

    files = [f for f in glob.glob(path + "**/*.csv", recursive=True)]

    if not full:
        files = [file.replace(path + "/", "") for file in files]
        files = [file.replace(".csv", "") for file in files]

    return files

In [21]:
OBJECTS_BY_PAGE = 24
MAX_PAGE = 400

SAVING_PATH = "/Users/lgmoneda/Documents/datasets/rijks/"
FOLDER = "photomechanical print"
directory = "{}{}".format(SAVING_PATH, FOLDER)
directory_metadata = "{}metadata".format(SAVING_PATH)

In [22]:
directory

'/Users/lgmoneda/Documents/datasets/rijks/photomechanical print'

In [23]:
if not os.path.isdir(directory):
    os.makedirs(directory, exist_ok=True)
    
if not os.path.isdir(directory_metadata):
    os.makedirs(directory_metadata, exist_ok=True)    

In [24]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
driver.implicitly_wait(5)

https://www.rijksmuseum.nl/en/collection/SK-A-3956

In [25]:
directory

'/Users/lgmoneda/Documents/datasets/rijks/photomechanical print'

In [26]:
def save_image_from_brick(brick, directory):
    file_name = brick.find("a", {"class": "link-block"})["href"].split("/")[-1] + ".jpg"
    
    image_src = brick.find("img", {"class": "lazy-image"})["data-src"].split("=w")[0]
    image_src += "=w1000"
    
    img = requests.get(image_src, stream=True)
    
    with open(directory + "/" + file_name, "wb") as f:
        shutil.copyfileobj(img.raw, f)
        
    return image_src

In [27]:
def trim_space_exclude_special_characters(text):
    return text.replace("\n", "").strip()

In [28]:
def get_meta_data_from_brick(brick):
    object_id = brick.find("a", {"class": "link-block"})["href"].split("/")[-1]
    title = brick.find("a")["aria-label"]
    subtle = brick.find("p", {"class": "text-subtle"}).text
    artist = trim_space_exclude_special_characters(subtle.split(",")[0])
    date = trim_space_exclude_special_characters(subtle.split(",")[-1]    )

    return {"id": [object_id],
            "title": [title],
            "artist": [artist],
            "date": [date]}

In [29]:
def save_meta_data_from_dict(metadata_dict, directory):
    df = pd.DataFrame.from_dict(metadata_dict)
    df.to_csv(directory + "/" + metadata_dict["id"][0] + ".csv")

In [30]:
#MAX_PAGE = 500
object_type = FOLDER
MIN_PAGE = 1

In [31]:
for page in range(MIN_PAGE, MAX_PAGE+1):
    print(page)
    downloaded = get_all_files_in_a_folder("metadata")
    
    url_template = "https://www.rijksmuseum.nl/en/search?f=1&p={}&ps={}&type={}&st=Objects&ii=0"
    page_link = url_template.format(page, OBJECTS_BY_PAGE, object_type)
    driver.get(page_link)
            
    soup = BeautifulSoup(driver.page_source)
    content = soup.find_all("figure", {"class": "brick"})

    for brick in content:
        try:
            metadata_dict = get_meta_data_from_brick(brick)
            if metadata_dict["id"] not in downloaded:
                metadata_dict["type"] = [object_type]
                save_meta_data_from_dict(metadata_dict, directory_metadata)
                save_image_from_brick(brick, directory)
        except:
            pass
driver.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [32]:
def aggregate_all_metadata(folder):
    
    files = get_all_files_in_a_folder(folder, full=True)

    all_dataframes = []
    for file in files:
        all_dataframes.append(pd.read_csv(file))
        
    return pd.concat(all_dataframes)

In [33]:
metadata_df = aggregate_all_metadata("metadata")

In [34]:
metadata_df.drop(columns=["Unnamed: 0"], inplace=True)

In [35]:
metadata_df.reset_index(drop=True, inplace=True)

In [36]:
metadata_df.to_csv(SAVING_PATH + "metadata.csv")

## Compress

In [20]:
SAVING_PATH

'/Users/lgmoneda/Documents/datasets/rijks/'

https://code.tutsplus.com/tutorials/compressing-and-extracting-files-in-python--cms-26816

In [21]:
directory

'/Users/lgmoneda/Documents/datasets/rijks/drawing'

In [22]:
# import os
# import zipfile
 
# fantasy_zip = zipfile.ZipFile(SAVING_PATH + "{}.zip".format(FOLDER), 'w')
 
# for folder, subfolders, files in os.walk(directory): 
#     for file in files:
#         if file.endswith('.jpg'):
#             fantasy_zip.write(os.path.join(folder, file), file, compress_type = zipfile.ZIP_DEFLATED)
#             #fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(file), SAVING_PATH), compress_type = zipfile.ZIP_DEFLATED)
 
# fantasy_zip.close()